In [ ]:
import MA4Star_Strategy as MA4
import os
import mplfinance as mpf
import matplotlib.pyplot as plt

In [ ]:
ticker = "0005.HK"
window_days = 55
alpha = 0.5
star_lbdays=0
predict_days=5
data_label = MA4.data_label(ticker, window_days, alpha, predict_days, star_lbdays)
print(data_label)

In [ ]:
batch_size = 32
epochs = 30
model_label = MA4.model_label(data_label, "CNN", epochs, batch_size)
print(model_label)
model = MA4.retreive_model(model_label)
if model is not None:
    model.summary()

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime

In [ ]:
period=f"{window_days+1}d"
print(period)
enddate = date(2025, 7, 3)
startdate = enddate - timedelta(days=window_days+1)
print(startdate, ' , ', enddate)
data = yf.download(ticker, start=startdate, end=enddate, progress=False)

In [ ]:
data.columns = [col[0] for col in data.columns]
display(data.shape)
display(data.head(2))
display(data.tail(2))

In [ ]:
# Calculate EMAs
data = MA4.calculate_emas(data)
data['EMStrend'] = None
data['Labels'] = None
data['RisingStar'] = None
data['EveningStar'] = None
# Convert to numeric type with np.nan for None
data['RisingStar'] = pd.to_numeric(data['RisingStar'], errors='coerce')
data['EveningStar'] = pd.to_numeric(data['EveningStar'], errors='coerce')
data = MA4.calulate_Alltrend(data, 0, alpha) 
data.head()

In [ ]:
# find Rising Star and Evening Star patterns
data = MA4.find_RE_star(data, 0, star_lbdays)
display(data.head())

In [ ]:
display(len(data))
chart_data = data.iloc[len(data)-window_days:]
print(chart_data.index[0], " -- ", chart_data.index[-1], ' length=', len(chart_data))
# Detect Rising Star and Evening Star patterns
rising_star_markers = chart_data['RisingStar']
evening_star_markers = chart_data['EveningStar']
# Build additional plots list, only include scatter plots if markers exist
ap = [
    mpf.make_addplot(chart_data['EMA_5'], color='red', width=1),
    mpf.make_addplot(chart_data['EMA_10'], color='blue', width=1),
    mpf.make_addplot(chart_data['EMA_20'], color='green', width=1),
    mpf.make_addplot(chart_data['EMA_30'], color='yellow', width=1)
]
# Only add scatter plots if there are non-NaN markers
if rising_star_markers.notna().any():
    # Ensure the series contains only numeric or np.nan values
    rising_star_markers = pd.to_numeric(rising_star_markers, errors='coerce')
    print(f"Adding Rising Star markers for window {i} is ")
    ap.append(mpf.make_addplot(rising_star_markers, type='scatter', marker='^', color='cyan', markersize=100))
if evening_star_markers.notna().any():
    evening_star_markers = pd.to_numeric(evening_star_markers, errors='coerce')
    print(f"Adding Evening Star markers for window {i} is ")
    ap.append(mpf.make_addplot(evening_star_markers, type='scatter', marker='v', color='magenta', markersize=100))
start_d = chart_data.index[0].strftime('%Y%m%d')
end_d = chart_data.index[-1].strftime('%Y%m%d')
chart_path = os.path.join("charts", f"chart_{ticker}_{start_d}_{end_d}.png")
print(chart_path)
try:
    fig, ax = mpf.plot(chart_data, type='candle', addplot=ap, savefig=chart_path, warn_too_much_data=1000, returnfig=True)
    plt.close(fig)  # Close the figure to free memory
    plt.close('all')  # Close all figures to prevent memory accumulation
except Exception as e:
    print(f"Error generating chart {chart_path}: {e}")


In [ ]:
img = MA4.preprocess_image(chart_path)
print(img.shape)
X_test = np.array([img])
print(X_test.shape)

In [ ]:
y_pred = model.predict(X_test)
print(y_pred)
y_pred_classes = np.argmax(y_pred, axis=1)
print(y_pred_classes)

In [ ]:
print(y_pred_classes)
print(MA4.LABEL_L[y_pred_classes[0]])